In [9]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings.cpu().numpy())

Sentence embeddings:
[[ 0.14317986 -0.23076145 -0.0139477  ...  0.03993536  0.10085303
  -0.19941317]
 [ 0.03999372 -0.20410602 -0.01313243 ...  0.02634653 -0.20096192
  -0.14515458]]


In [13]:
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloomz-7b1-mt')

In [28]:
sentences="""
AWS 文档中提到: Q：在中国区是否可用？
A： 目前没有落地中国区的时间表，已经在以下区域推出：美国东部（弗吉尼亚州北部）、美国东部（俄亥俄州）、美国西部（俄勒冈州）、亚太地区（首尔）、亚太地区（新加坡）、亚太地区（悉尼）、亚太地区（东京）、欧洲地区（法兰克福）、欧洲地区（爱尔兰）、欧洲地区（伦敦）和欧洲地区（斯德哥尔摩）

Q: 为什么在合成的小数据集上第一次查询的时候需要几分钟才返回？
A：第一次查询的时候是因为调度和拉起资源的影响，一般第二次查询就会变快。 但过一段时间后，资源释放后这个问题又会出现。在资源拉起期间是不进行收费的。

Q：能支持多大规模数据的查询？查询速度怎么样？
A：能支持TB/GB级数据的查询。 一般查询延迟为几十秒到几分钟。默认计算容量为32 CRPUs, 目前这个默认计算容量不可设置，但是roadmap中未来打算让用户可以进行设置。(Slack中Ryan 提到，如果引擎中任务有积压，它能够scale up）

Q: AWS Clean Rooms 是如何计费的？
A: 按照CRPU-hour单价进行计费，每个查询默认计算容量为32 CRPUs。 金额 = (0.125 hours x 32 CRPUs * $0.656 per CRPU-hour) , 有1分钟的最小计费时间。头12个月内，会有9CRPU hours的免费额度。

Q: AWS Clean Rooms 从哪里可以看到CRPU-hours的用量？
A: AWS Clean Rooms 本身的workspace中无法查看，可以在AWS Billing/Bills 中查看Usage Quantity得到该信息。

Q: 目前可以支持什么数据源的接入？ 
A: 目前只支持S3，其他数据源近期没有具体计划。

Q: 一个协作中，最大的并发查询数是多少？
A: 5个

Q: 我们如何说服客户相信洁净室的安全性和正确性？我在这里的大多数合规计划中都看不到Clean Rooms， https://aws.amazon.com/compliance/services-in-scope/ ？
A: 正在加入这些合规计划的进程中。


使用方法与限制相关:

Q: 数据源必须在AWS上么？
A: 对，目前必须在AWS上，而且必须是同一个region。

Q: 数据是如何进入到S3？
A: 需要数据的持有者，把数据上传到S3上，然后再用Glue爬去下，拿到表的schema。这样才能关联到AWS CleanRooms。

Q: 这些安全控制权限只是作用与分析么？能够改动它方的数据么？
A: 对，只作用于分析，不能修改对方数据

Q: 协作方的数据会移动么？协作方的原始数据会集中到Clean rooms 吗？
A: 在联合分析获取他方数据时数据存在移动，但协作方的原始数据并不会存住在clean room内，clean room并不是一个物理存贮空间。

Q: 是否发起者和数据贡献者都会被收费？
A: 是单方收费，只有查询的接收方会进行收费。

Q: 数据贡献方的S3， 会产生API会产生调用次数收费么？
A: 会， Glue Data Catalog API 的调用也会被收费， 如果加密数据用了KMS-CMK也会被相应的收费。

Q: 是否有一个强约束，两方的数据中一定要一个join字段才能够进行分析？
A: List 和Aggregation两种不同的分析规则下有区别， List 只能支持重合用户的，所以必须要有关联字段。Aggregation可以支持你仅仅去查询对方的数据，这种情况下，是可以不指定关联字段的。

Q: 如果已经在用Athena，S3桶中已经有数据了，是否能基于S3中这个数据就地加入AWS Clean Room?
A: 是的，就地就能加入clean room, 这个S3桶就是一般的S3桶，并没有任何特殊。但这个S3路径不能注册到AWS Lake Formation中。

Q: 是否能通过SDK也就是代码来调用Clean room的联合分析
A: 可以，可以参考代码 https://gitlab.aws.dev/rmalecky/aws-clean-rooms-notebooks/-/blob/main/single_collaborator_aggregation.ipynb

Q: 在输出分析结果的时候，能否按照字段进行分区？
A: 目前不能

Q: 最大的参与方是多少？ 如果超过了限制怎么办？
A: 目前5个参与方为最大限制，这个是软性的限制，slack频道中有披露最大支持的硬限制为10.


数据加密相关：

Q: AWS clean rooms 用的什么加密算法？
A: C3R，是aws开源加密代码库。提供了C3R Client(一个可执行的Jar包)，目前仅支持对csv和parquet文件格式进行加密，后续可能会支持更多格式。 由于clean room把所有的字段分成三种类型： 指纹列(fingerprint column), 密封列(sealed column), 明文列(cleartext column), 他们的加密方式有所不同，C3R client 会使用AES-GCM加密算法对sealed字段进行加密，会使用HMAC(Hash-based Message Authentication Code)来对fingerprint字段进行加密。

Q: 加密过程中有密文落地么？如果有，密文存在哪里？
A：由于C3R是客户端加密，所以clean room 关联S3中的数据已经是加密后的密文。

Q: 数据上传到S3的过程中，有两种加密方式Server Side 和 Client Side加密，如果客户的安全等级比较高，在数据上传之前做了加密，再想交给clean room 去处理，之前提到的C3R的加密方式，具体是一个怎么样的流程呢？
A: 首先Clean room 对于Server Side的加密是透明的，无需额外处理。Clean rooms 不支持S3的客户端加密，必须采用C3R客户端进行加密，加密完成以后把数据上传到S3桶，后续流程和不加密的流程是一致的。 加密这步需要比较多的手工操作，包括：

* 加密前需要创建好collaboration(协作), 得到collaboration_id后续在加密中需要提供
* 利用openssl 生成32位密钥并分享给其他协作方。
* 加密过程中需要指定哪些字段为指纹列(fingerprint column), 密封列(sealed column), 明文列(cleartext column)


Q: AWS Clean Room在安全计算方面，应该归属哪一类？
A: (待补充)

Q: 是否所有的字段都可以进行加密？
A: 对于sealed和 fingerprint字段，只有string字段类型被支持。对于csv文件，C3R的客户端处理任何值都作为UTF-8编码的文本，加密前不会做任何其他的前置处理。对于parquet文件，对sealed和 fingerprint字段，如果出现非string的字段，会直接报错。C3R 客户端不能处理parquet中的复杂字段比如struct。

Q: 如果要对某个字段进行求和，求平均的数据计算，是否可以加密？
A: 不能，只能作为cleartext明文列

Q: 如果想要通过某些字段进行where过滤，这些字段应该是什么类型？
A: 只能是cleartext明文列

Q: C3R客户端是否有实现任何non-standard的加密算法?
A: 基本都是标准化的算法，除了一个HKDF(一种密钥推导函数)的实现(来自RFC5869), 但是使用的是java标准加密库中的MAC算法。


宏观问题：

Q: AWS Clean rooms 与 AMC的区别与联系是什么？
A: AMC是一个专门服务与Amazon Ads的clean room应用，它是并且将持续是唯一的服务于Amazon Ads客户的应用服务。AWS Clean Rooms 是一个云分析服务，会服务于各个行业的数据合作需求。2023年, AMC 将会把自己的查询引擎和计算基础设置迁移到AWS Clean Rooms服务，将会帮助AMC更方便的服务于客户(他们将不再需要把自己第一方数据上传到AMC，在AWS S3上即可使用).

Q: AWS Clean rooms 与 其他的clean room服务商的区别？
A: AWS Clean rooms 覆盖的客户范围更广。而其他的服务商客户范围相对小，需要把数据移动到他们的平台上。AWS Clean rooms则无需数据移动。(Bastion 2021年announce的时候，其他的云厂商比如(google cloud 和 microsoft azure) 还没有通用场景的clean room solution，snowflake 有，功能上有区别，只允许data provider 提供预先固定的SQL，Bastion灵活性更好。snowflake要求数据必须进入他们的数仓，aws在S3 即可)。

Q: 有哪些典型的应用场景？
A: 包含多个行业，下面提供部分参考

* 广告营销领域：
    * 需要进行广告营销活动，流量平台方需要给广告主或者营销方他们的广告点击数据和展现数据进行分析，但是不能提供用户级别的信息。
    * 典型客户：营销方或广告主 P&G, Barclays，媒体-流量平台 Amazon Ads, Comcast, NBC Universal
* 零售领域
    * 银行和零售商需要获取他们的重叠用户，用于进行联合的市场活动，但不要把客户的其他信息暴露给彼此。（比如非场合用户）
* 医疗健康领域 (结合之前的HCLS Slides， 并不是来自Bastion)
    * 药厂和医院之间，药厂需要医院的病历数据； 药厂和外包的研发机构之间需要进行数据的共享。
    * 典型客户：Change Healthcare, AstraZeneca 
* 其他领域的一些典型的客户
    * 数据服务商 Foursquare ，Nielsen， IRI， 
    * 其他 Cars.com
    
内部信息(来自Bastion PRFAQ(2021年))[Internal Only]:

Q: AWS Clean Rooms 的 data catalog 是如何实现的？ data sharing permission 是如何实现的？
A: 都是利用了AWS Lake formation, AWS Clean Rooms 里 SQL中字段级别的限制约束，是通过一种new class of AWS Lake formation permission 来实现的。

Q: 可以在哪些地方进行Clean room的联合分析？
A: 可以在clean room 的 workspace， 也可以在Redshift workspace （Note: 从目前发布产品文档上并没有，但是说明背后的引擎就是redshift severless)

Q: 数据提供方如果对联合分析的收益方进行收费，或者实现一个数据授权的合同？
A: 需要通过AWS Data Exchange 来进行 （Note: 目前AWS Clean Rooms并没有体现）

Q: AWS Clean Rooms 与 AWS Data Exchange 是什么关系？
A: AWS Clean Rooms 可以通过AWS Data Exchange 去浏览和寻找可用数据的合作方。 他是AWS Data Exchange的更近一步的服务，提供了可控(多种约束限制)和可审计的数据合作方式

Q: AWS Clean Rooms中是否支持视图？
A: 允许客户在clean room 创建视图，并且在AWS Clean Rooms中保存物化视图，一旦退出协作，AWS Lake formation permission 将会被撤销，这些物化视图会被删除。（Note: 目前AWS Clean Rooms并没有体现）

Q: 如果数据合作方没有aws account，能否支持？
A: 目前这个版本不支持，后续的版本可能会考虑（NBC Universal 希望对于没有aws账号另外一方的数据可用）

Q: 是否能够支持这个协作中，仅仅允许指定运行固定的SQL？
A: 可以，可以利用query template来做（Note: 目前AWS Clean Rooms并没有体现）

Q: AWS Clean Rooms可以让数据贡献者提供一些样例数据进行预览么？
A: 可以这么做，可以提供一些没有任何约束的示例数据给用户（Note: 目前AWS Clean Rooms并没有体现）

Q: 当一个数据贡献者的数据发生更新后会怎么样？
A: 它是一种live的共享，任何更新会立刻反映到联合分析的结果中

"""

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
print(encoded_input)

{'input_ids': tensor([[  189,    36, 15799,  ..., 57427,   773,   603]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [29]:
encoded_input.input_ids.size()

torch.Size([1, 2455])

In [5]:
!pip install transformers


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
